# SPRINT - 2

## Item - Levantamento dos cálculos dos indicadores.

## Item - Levantamento do procedimento de consolidação dos dados de produção e apontamentos.

## Item - Desenvolvimento do processamento de Limpeza, Formatação e Classificação dos dados, e gravação na "Refined Data".

<br>
<br>

Autor.: Sérgio C. Medina

#### Declaração dos Pacotes, Libs ou Classes utilizadas no processo.

In [2]:
# Declaração dos Pacotes, Libs ou Classes utilizadas no processo.
import os
import io
import math
import pandas as pd
import gcsfs
import pyarrow
import pyarrow.parquet as pq
from google.cloud import storage
from datetime import datetime, timedelta

import sys
sys.path.append('../../pods')

# configurando variavel de ambiente com o arquivo de credenciais para conexão GCP
#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "..\..\secrets\edc-igti-smedina-4920e12ac565.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/sergiomedina/Downloads/edc-igti-smedina-4920e12ac565.json"

# Funções de integração com o Cloud Storage
from utilGCS import utilGCS
gcs = utilGCS(bucketName='edc-pa-i4-data')

# Funções diversas de manipulação de dados
from utilFuncs import utilFuncs
func = utilFuncs()


#### Dados na "raw-data-zone" pasta "DATAOP"

In [ ]:
dtexec = "2021-11-10"
source = "dataop"
folder = f"raw-data-zone/{source}"

df_dataop = gcs.read_csv_to_df(folder=folder, dtexec=dtexec, sep=";")

if df_dataop is not None:
    # Limpa os dados nulos
    df_dataop.dropna(inplace=True)

    # Define os tipos de cada coluna
    df_dataop['OP'] = df_dataop['OP'].astype(str)
    df_dataop['CODMAT'] = df_dataop['CODMAT'].astype(str)
    df_dataop['LOTEFAB'] = df_dataop['LOTEFAB'].astype(str)
    df_dataop['DTINI'] = pd.to_datetime(df_dataop['DTINI'])
    df_dataop['DTFIM'] = pd.to_datetime(df_dataop['DTFIM'])
    df_dataop['QTDPLAN'] = df_dataop['QTDPLAN'].astype(int)

    # Definindo a Data de Produção DTPROD
    df_dataop.insert(0, 'DTPROD', None)
    df_dataop['DTPROD'] = df_dataop.apply(lambda row:func.compate_dtprod(row['DTINI'], row['DTFIM']),axis=1)
    df_dataop['DTPROD'] = df_dataop['DTPROD'].astype(str)

    # save parquet - gcp
    # https://gist.github.com/lpillmann/fa1874c7deb8434ca8cba8e5a045dde2
    # https://blog.datasyndrome.com/python-and-parquet-performance-e71da65269ce

    # Verifica se existe arquivos no path para deletar
    gcs.delete_blob(pathName=f"processing-zone/{source}/DTPROD={dtexec}")

    # Gravando na Processing-Zone
    gcs.write_pandas_to_parquet(path=f"edc-pa-i4-data/processing-zone/{source}", df=df_dataop, partitionCols=['DTPROD'])


In [ ]:
df_dataop.dtypes

In [ ]:
# Teste de leitura do Parquet
source = "dataop"
#df = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}")
df = gcs.read_parquet_to_pandas(path=f"edc-pa-i4-data/processing-zone/{source}", filters=[('DTPROD', '=', '2021-11-08')])

df

#### Dados na "raw-data-zone" pasta "DATAPROD"

In [8]:
dtexec = "2021-11-09"
lineid = "101"
source = "dataprod"
folder = f"raw-data-zone/{source}"

# Carregando os dados do arquivo da data e linha informados
df_dataprod = gcs.read_csv_to_df(folder=folder, dtexec=dtexec, lineprod=lineid, sep=";")

if df_dataprod is not None:

    # verificando se existe dados do dia posterior para completar o turno 3
    nextDay = datetime.strptime(dtexec, "%Y-%m-%d") + timedelta(days=1)
    print("dtexec:", dtexec, " next:", nextDay)    
    df = gcs.read_csv_to_df(folder=folder, dtexec=nextDay.strftime("%Y-%m-%d"), lineprod=lineid, sep=";")
    if df is not None:
        df_dataprod = df_dataprod.append(df, sort=False, ignore_index=True)

    # Limpa os dados nulos
    df_dataprod.dropna(inplace=True)

    # Alterando o delimitador decimal de "," para "."
    df_dataprod['TOTMIN']=df_dataprod.TOTMIN.str.replace(',','.')

    # Define os tipos de cada coluna
    df_dataprod['OP'] = df_dataprod['OP'].astype(str)
    df_dataprod['LINE'] = df_dataprod['LINE'].astype(int)
    df_dataprod['LINE'] = df_dataprod['LINE'].astype(str)
    df_dataprod['TIMESTAMP'] = pd.to_datetime(df_dataprod['TIMESTAMP'])
    df_dataprod['BATCH'] = df_dataprod['BATCH'].astype(str)
    df_dataprod['TIMER'] = df_dataprod['TIMER'].astype(str)
    df_dataprod['TOTMIN'] = df_dataprod['TOTMIN'].astype(float)
    df_dataprod['STSID'] = df_dataprod['STSID'].astype(int)
    df_dataprod['STSDS'] = df_dataprod['STSDS'].astype(str)
    df_dataprod['PC'] = df_dataprod['PC'].astype(float)
    df_dataprod['GOOD'] = df_dataprod['GOOD'].astype(float)
    df_dataprod['REJECT'] = df_dataprod['REJECT'].astype(float)

    # Definindo a Data de Produção DTPROD
    df_dataprod.insert(0, 'DTPROD', None)
    df_dataprod['DTPROD'] = df_dataprod.apply(lambda row:func.calc_dtprod(row['TIMESTAMP']),axis=1)
    df_dataprod['DTPROD'] = df_dataprod['DTPROD'].astype(str)
    
    # Definindo a ID do Turno de Produção IDTURNO
    df_dataprod.insert(1, 'IDTURNO', 0)
    df_dataprod['IDTURNO'] = df_dataprod.apply(lambda row:func.calc_idturno(row['TIMESTAMP']),axis=1)
    df_dataprod['IDTURNO'] = df_dataprod['IDTURNO'].astype(int)

    # Seleciona apenas o periodo referente ao DTPROD
    df_dataprod = df_dataprod.loc[ df_dataprod['DTPROD'] == dtexec ]

    # reindex
    df_dataprod = df_dataprod.reset_index(drop=True)

    # Verifica se existe arquivos no path para deletar
    gcs.delete_blob(pathName=f"processing-zone/{source}/DTPROD={dtexec}")

    # Gravando na Processing-Zone
    gcs.write_pandas_to_parquet(path=f"edc-pa-i4-data/processing-zone/{source}", df=df_dataprod, partitionCols=['DTPROD'])


dtexec: 2021-11-09  next: 2021-11-10 00:00:00
SEARCHING TO DELETE: processing-zone/dataprod/DTPROD=2021-11-09
SUCCESS: edc-pa-i4-data/processing-zone/dataprod


In [ ]:
df_dataprod.dtypes

In [16]:
#df.groupby(['col1', 'col2']).agg({'col3':'sum','col4':'sum'})
df_sts = pd.DataFrame(df_dataprod.groupby(
    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH', 'STSID', 'STSDS']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum'
    }
)).reset_index()


df = df_dataprod.copy()

df['TMIN'] = df['TIMESTAMP']
df['TMAX'] = df['TIMESTAMP']
df.drop(['TIMESTAMP'], axis=1, inplace=True)

df = pd.DataFrame(df.groupby(
    ['DTPROD', 'IDTURNO', 'LINE', 'OP', 'BATCH']
).agg(
    {
        'TOTMIN': 'sum',
        'PC': 'sum',
        'GOOD': 'sum',
        'REJECT': 'sum',
        'TMIN': 'min',
        'TMAX': 'max'
    }
)).reset_index()


df['TMAXLIMIT'] = df.apply(lambda row:func.end_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMAXLIMIT'] = pd.to_datetime(df['TMAXLIMIT'])
df['TMAXDIFF'] = (df['TMAXLIMIT']-df['TMAX']).dt.seconds/60
df['WIP'] = df.apply(lambda row:func.round_down(row['TMAXDIFF'], 0),axis=1)

df['TMINLIMIT'] = df.apply(lambda row:func.start_time(row['DTPROD'], row['IDTURNO']),axis=1)
df['TMINLIMIT'] = pd.to_datetime(df['TMINLIMIT'])
df['TMINDIFF'] = (df['TMIN']-df['TMINLIMIT']).dt.seconds/60

df['DIFF'] = df['TMAXDIFF'].diff().fillna(0)

df['TOTMINADJUSTED'] = round(df['TOTMIN'] + df['DIFF'],0)

df.drop(['TMAX', 'TMAXLIMIT', 'TMAXDIFF'], axis=1, inplace=True)
df.drop(['TMIN', 'TMINLIMIT', 'TMINDIFF', 'DIFF'], axis=1, inplace=True)

# Tot. Tempo turno
df['TPTOTAL'] = 480

df['TOTMINADJUSTED'] = df.apply(lambda row:(row['TOTMINADJUSTED'] if row['TOTMINADJUSTED']<480 else 480),axis=1)

# Tot. Tempo = WORKING
df['TPWORKING'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==4) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

df['TPWORKING'] = df.apply(lambda row:(row['TPWORKING'] if row['TPWORKING']<=row['TOTMINADJUSTED'] else row['TOTMINADJUSTED']),axis=1)

# Tot. Tempo = TPSTOPPLAN
df['TPSTOPPLAN'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

# Tot. Tempo = TPNOALLOC
df['TPNOALLOC'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID==0) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)    


# Tempo Programado para produzir
df['TPPROG'] = df['TPTOTAL'] - (df['TPNOALLOC']+df['TPSTOPPLAN'])


# Temp Ociosidade
df['TPIDLE'] = df.apply(
    lambda row:df_sts.where(
    (df_sts.STSID>=1) &
    (df_sts.STSID<3) &
    (df_sts.DTPROD==row['DTPROD']) &
    (df_sts.IDTURNO==row['IDTURNO']) &
    (df_sts.LINE==row['LINE']) &
    (df_sts.OP==row['OP']) &
    (df_sts.BATCH==row['BATCH']) 
    ).agg(
        {'TOTMIN':'sum'}
    )
    ,axis=1)

df['TPIDLE'] = ((df['TPPROG']-(df['TPWORKING'] + df['TPIDLE']))+df['TPIDLE'])

# Unidades - Produção Teórica
df['PCTHEOR'] = df['TPPROG'] * 1

# Unidades - Perda por performance
df['PCLOSS'] = df['PCTHEOR'] - df['PC']


# OEE Calc
# df['OEEDISP'] = df['TPWORKING'] / df['TPPROG']

# df['OEEPERF'] = df['PC'] / df['PCTHEOR']

# df['OEEQUAL'] = df['GOOD'] / df['PC']

# df['OEE'] = df['OEEDISP'] * df['OEEPERF'] * df['OEEQUAL']




In [17]:
df

,DTPROD,IDTURNO,LINE,OP,BATCH,TOTMIN,PC,GOOD,REJECT,WIP,TOTMINADJUSTED,TPTOTAL,TPWORKING,TPSTOPPLAN,TPNOALLOC,TPPROG,TPIDLE,PCTHEOR,PCLOSS
0,2021-11-09,1,101,211109L101,TB70395,474.52,350.0,314.0,36.0,5.0,475.0,480,341.24,0.0,0.0,480.0,138.76,480.0,130.0
1,2021-11-09,2,101,211109L101,TB70395,482.07,452.0,451.0,1.0,3.0,480.0,480,480.00,0.0,0.0,480.0,0.00,480.0,28.0
2,2021-11-09,3,101,211109L101,TB70395,477.85,445.0,440.0,5.0,5.0,480.0,480,477.85,0.0,0.0,480.0,2.15,480.0,35.0


In [11]:
df.agg(
    {
        'OEEDISP': 'prod',
        'OEEPERF': 'prod',
        'OEEQUAL': 'prod',
        'OEE': 'prod'
    }
)

OEEDISP    0.707732
OEEPERF    0.636565
OEEQUAL    0.885100
OEE        0.398753
dtype: float64

In [ ]:
0.879834 * 0.827490 * 0.877649